In [37]:
import numpy as numpy
from mpmath import *
mp.dps = 15; mp.pretty = True

####  1) We want to compute $$ \int u*u*C $$

Where $u$ is the depth averaged velocity, and $C$ is the depth averaged concentration

2) correct factor 0.6 by computing the correct value with non-singular velocity profile (SEE MIEDEMA 5.3.44)

3) compute ratio of concentration at z=0 and average concentration (DONE)

In [38]:
# Flow average quantities

h      = 10.00  # Flow thickness, in meters
uC_avg =  0.01  # Depth averaged product of velocity and concentration (u*sum(C_i))

In [39]:
# particles average quantities

C_avg = numpy.zeros(2) # initialize vector
C_avg[0] = 0.050    # depth-averaged volumetric fraction
C_avg[1] = 0.006    # depth-averaged volumetric fraction
np = len(C_avg)     # variable for length of Concentration vector

print(C_avg) # check assignment

[0.05  0.006]


In [40]:
# particles parameters
settl_vel = numpy.zeros(2) # initialize vector

settl_vel[0] = 0.05  # [m/s]
settl_vel[1] = 0.5   # [m/s]

print(settl_vel) # check assignment

[0.05 0.5 ]


In [41]:
# fixed parameters
k    = 0.4             # Von Karman constant
k_s  = 0.1             # substrate roughness [m] 
beta = 1.0             # ratio of momentum and mass diffusivity (Schmidt number)
friction_coeff = 0.01  # friction coefficient

$H_{crit}$ is a critical thickness above which there is a constant free stream velocity. 

Below $H_{crit}$ we only have the logarithmic velocity profile.

$H_{crit}$ is a function of $k$, the Von Karmann Constant, $k_s$, the substrate roughness,  and $C_f$, the coefficient of friction.


$H_{crit_{rel}}$ is the non-dimensional crtitical thickness, which depends on $k$ and the friction coefficient $C_f$ only

$$ a = \frac{k}{\sqrt{C_f}}+1.0$$


$$ H_{crit_{rel}} = \frac{1}{30} * \left[\frac{-a}{\mathcal{W}\left(-a*e^{-a}\right)} -1.0 \right] $$

where $\mathcal{W}$ is the $\textit{lambert W}$ or $ \textit{product log}$ function

In [42]:
# We need something to compute the lambert function in python

from scipy.special import lambertw # imports the lambert function and returns complex result

In [45]:
a = k/numpy.sqrt(friction_coeff)+1.0
H_crit_rel = 1.0/30.0 * ( -a / lambertw(-a*numpy.exp(-a)) -1.0 )

print("Log profile thickness = ",H_crit_rel*k_s)

Log profile thickness =  (0.47441640531351936+0j)


The profile parameters depend on:  

# $\frac{h}{k_s}$

not on the absolute value of $h$.



In [46]:
h_rel = h/k_s

#### 

We search for $ h_{0_{rel}} $ such that the average integral between 0 and $h_{rel}$ is equal to 1
For $h_{rel} > H_{{crit}_{rel}}$ this integral is the sum of two pieces:
1) the integral between 0 and $h_{0_{rel}}$ of the log profile, and 


2) the integral between $h_{0_{rel}}$ and $h_{rel}$ of the costant profile

When $ h_{rel} <= H_{crit_{rel}}$ we have only the log profile and we have to rescale it in order to have the integral between o and $h_{rel}$ equal to 1. The factor used to scale the velocity is $u_{coeff}$

In [48]:
if h_rel > H_crit_rel:
    a = h_rel * k/numpy.sqrt(friction_coeff)
    b = 1.0/30.0 + h_rel
    c = 30.0

    # solve b*log(c*z+1)-z=a for z
    d = a/b-1.0/(b*c)
    
    # Then:
    h0_rel = -b*lambertw( -numpy.exp(d)/(b*c) ) - 1.0 / c
    u_coeff = 1.0
    
else:
    h0_rel = h_rel
    u_coeff = k/numpy.sqrt(friction_coeff)/( ( 1 + 1/(30*h_rel) )*numpy.log( 30.0*h_rel + 1.0 )-1.0)
    

In [50]:
h0 = h0_rel * k_s
b  = 30.0 / k_s
u_rel0 = u_coeff * numpy.sqrt(friction_coeff) / k * numpy.log( b*h0 + 1.0 )

# search convergence of u_avg
n = 200

# initial guess for average velocity
u_guess = uC_avg / sum(C_avg)

loop to compute the average velocity from average C(i) and average uC, 

### $$ = \frac{1}{h * int * \sum \left[u*C_i\right]} $$

In [76]:
import numpy as numpy
from mpmath import *
mp.dps = 15; mp.pretty = True

def avg_profiles(h,settl_vel,k,friction_coeff,uRhoAlpha_avg,rhoAlpha_avg,u_guess,beta,h0,b,u_coeff,u_rel0):
    
    # The input here are the depth-averaged velocity and the depth-averaged
    # values of rho*C(z). We compute the vertical profiles and the integral of 
    # u(z)*rho*C(z), and we compare this value with the desired one to update
    # the average velocity.

    # Shear velocity computed from u_guess
    u_star = u_guess * numpy.sqrt(friction_coeff)

    # Rouse numbers for the particle classes
    Pn = settl_vel / ( k * u_star )

    # number of particle classes
    np = len(settl_vel)

    # array for depth-averaged value of rho*u(z)*C(z)
    rho_uAlpha = numpy.zeros(np)

    rhoAlpha_int = numpy.zeros(np)
    
    
    for i in range(0,np-1):

        a = -(0.6*Pn[i]*beta)/h 

        int = ( ( numpy.exp(a*h0) -1.0 )/a + numpy.exp(a*h0)*(h-h0) )/h 

        alpha_rel_max = 1.0/int 

        # relative concentration alpha_rel at depth h0 (from the bottom)
        # alpha_rel is defined as alpha(z)/alpha_avg
        alpha_rel0 = alpha_rel_max * numpy.exp(a*h0) 

        # depth-averaged value of rho*C(z)
        # this results from the sum of the log region with thickness h0 and the 
        # constant region with thickness h-h0
        rhoAlpha_int[i] = rhoAlpha_avg[i] * ( alpha_rel_max * ( numpy.exp(a*h0) - 1.0 ) / a + (h-h0)*alpha_rel0 ) / h   

        # we compute the integral in the log region of u_rel(z)*C_rel(z), where 
        # u_rel is defined as u(z)/(u_guess*u_coeff*sqrt(friction_coeff)) 
        y = h0 
        int_h0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ei( -a*(y+1.0/b) ) ) / a 
        y = 0 
        int_0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ei( -a*(y+1.0/b) ) ) / a 

        # integral of u_rel(z)*C_rel(z) in the log region (0<=z<=h0)
        int_def = u_coeff * numpy.sqrt(friction_coeff) / k * alpha_rel_max * (int_h0-int_0) 

        # we add the contribution of the integral of the constant region, we
        # average by dividing by h and we multiply by the density of solid and
        # average concentration
        rho_uAlpha[i] = rhoAlpha_av[i] * ( int_def + ( h - h0 ) * alpha_rel0 * u_rel0 ) / h 


    rhoAlpha_new_avg = sum(rhoAlpha_int) 
    rho_uAlpha_new_avg = sum(rho_uAlpha) 

    u_new_avg = uRhoAlpha_avg / ( rho_uAlpha_new_avg) 
    rho_uAlpha_new_avg = u_new_avg * rho_uAlpha_new_avg 
    
    return([u_new_avg,rhoAlpha_new_avg,rho_uAlpha_new_avg])




In [77]:
for i in range(0,n):

    [u_new_avg,C_new_avg,uC_new_avg] = avg_profiles(h,settl_vel,k,friction_coeff,uC_avg,C_avg,
                                                    u_guess,beta,h0,b,u_coeff,u_rel0)

    if ( abs(u_guess-u_new_avg)/u_guess < 1.e-10 ):
        u_guess = u_new_avg
        break


    u_guess = u_new_avg

    print('u_guess = ', u_new_avg)

print('Iteration = ',i)


TypeError: unsupported operand type(s) for *: 'float' and 'method'

In [78]:
# average velocity
u_avg = u_new_avg 

# shear velocty
u_star = u_avg * numpy.sqrt(friction_coeff) 

# Rouse numbers for different particles
Pn = settl_vel / ( k * u_star ) 


## Check for average values, computed from u_avg and c_avg(i) 

# average velocity computed as integral of the profile
U_int = ( u_avg * ( u_coeff * sqrt(friction_coeff) / k * ( (h0 + 1.0/b)*log(b*h0+1)-h0 ) + (h-h0)*u_rel0 ) ) / h 

print('Depth-averaged u = ',U_int) 

uC_int = numpy.zeros(1,2) 
    

NameError: name 'u_new_avg' is not defined

In [90]:
for i in range(0,np-1):

    print('Rouse Number for solid class' ,i + 1, ' = ',Pn[i])

    a = -(0.6*Pn[i]/beta)/h;

    int = ( ( numpy.exp(a*h0) -1.0 )/a + numpy.exp(a*h0)*(h-h0) )/h;

    C_rel_max = 1.0/int;

    C_rel0 = C_rel_max * numpy.exp(a*h0);

    # average concentration of particle class i
    C_int = C_avg(i) * ( C_rel_max * ( exp(a*h0) - 1.0 ) / a + (h-h0)*C_rel0 ) / h;  

    print('Depth-averaged Concentration ',i,' = ',C_int)
    
    y = h0;
    int_h0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ei( -a*(y+1.0/b) ) ) / a;
    y = 0;
    int_0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ei( -a*(y+1.0/b) ) ) / a;

    int_def = u_coeff * numpy.sqrt(friction_coeff) / k * C_rel_max * (int_h0-int_0);

    # average of u*C(i)
    uC_int[i] = u_avg * C_avg(i) * ( int_def + ( h - h0 ) * C_rel0 * u_rel0 ) / h;

uC_int_tot = sum(uC_int);

print('Depth-averaged u*C = ',uC_int_tot)

Rouse Number for solid class 1  =  1


TypeError: unsupported operand type(s) for /: 'float' and 'method'

In [71]:
Pn = [1,2,3]
beta = 1
np = 4
h = 1

for i in range(0,np-1):
    a = -(0.6*Pn[i]*beta)/h 
    print(a)

-0.6
-1.2
-1.7999999999999998


In [85]:
def test(x):
    y = x**2
    return(y)

test(2)

[a,b,c] = [test(1),test(2),test(3)]
f = [a,b,c]
print(f)



[1, 4, 9]
